<a href="https://colab.research.google.com/github/chhendley/chhendley_python_projects/blob/main/heart_attach_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#consider further shrinking the focused_heart DF further to increase accuracy
#tune neurons to achieve 93% or higher accuracy

In [1]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install scikeras
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.1 MB/s eta 0:00:00


In [ ]:

# first neural network with keras make predictions
import keras_tuner
from tensorflow import keras
from keras_tuner import RandomSearch, GridSearch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import keras_tuner
from tensorflow import keras
from keras_tuner import RandomSearch, GridSearch
import keras
from keras.layers.core.dense import utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.saving.legacy.saving_utils import model_call_inputs
from keras.layers import BatchNormalization, LayerNormalization
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from keras.layers import Input, Dense
from keras.layers import Dropout, Activation

In [ ]:
# load the dataset
url='https://github.com/chhendley/chhendley_python_projects/blob/main/heart.csv'
heart=pd.read_csv('heart.csv')

In [ ]:
print(heart['output'].value_counts(ascending=True))

0    138
1    165
Name: output, dtype: int64


In [ ]:
#print correlation matrix to find which attributes are strongly correlated with output
heart.corr()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
age,1.000000,-0.098447,-0.068653,0.279351,0.213678,0.121308,-0.116211,-0.398522,0.096801,0.210013,-0.168814,0.276326,0.068001,-0.225439
sex,-0.098447,1.000000,-0.049353,-0.056769,-0.197912,0.045032,-0.058196,-0.044020,0.141664,0.096093,-0.030711,0.118261,0.210041,-0.280937
cp,-0.068653,-0.049353,1.000000,0.047608,-0.076904,0.094444,0.044421,0.295762,-0.394280,-0.149230,0.119717,-0.181053,-0.161736,0.433798
trtbps,0.279351,-0.056769,0.047608,1.000000,0.123174,0.177531,-0.114103,-0.046698,0.067616,0.193216,-0.121475,0.101389,0.062210,-0.144931
chol,0.213678,-0.197912,-0.076904,0.123174,1.000000,0.013294,-0.151040,-0.009940,0.067023,0.053952,-0.004038,0.070511,0.098803,-0.085239
fbs,0.121308,0.045032,0.094444,0.177531,0.013294,1.000000,-0.084189,-0.008567,0.025665,0.005747,-0.059894,0.137979,-0.032019,-0.028046
restecg,-0.116211,-0.058196,0.044421,-0.114103,-0.151040,-0.084189,1.000000,0.044123,-0.070733,-0.058770,0.093045,-0.072042,-0.011981,0.137230
thalachh,-0.398522,-0.044020,0.295762,-0.046698,-0.009940,-0.008567,0.044123,1.000000,-0.378812,-0.344187,0.386784,-0.213177,-0.096439,0.421741
exng,0.096801,0.141664,-0.394280,0.067616,0.067023,0.025665,-0.070733,-0.378812,1.000000,0.288223,-0.257748,0.115739,0.206754,-0.436757
oldpeak,0.210013,0.096093,-0.149230,0.193216,0.053952,0.005747,-0.058770,-0.344187,0.288223,1.000000,-0.577537,0.222682,0.210244,-0.430696


In [ ]:
heart_even = heart.drop(heart.index[130:164])
heart_even = heart_even.drop(heart_even.index[295:])
display(heart_even.shape)

(269, 14)

In [ ]:
#drop weakest correlations
focused_heart = heart_even.drop(['trtbps',	'chol',	'fbs'], axis=1)
#focused_heart = heart_even.drop(['trtbps',	'chol',	'fbs','restecg', 'age', 'sex'], axis=1)
display(focused_heart.shape)

(269, 11)

In [ ]:
X = heart.loc[:, 'age':'thall']
y = heart.loc[:,'output']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=104,test_size=0.25, shuffle=True)

In [ ]:
def create_baseline():
 # create model
 model = Sequential()
 model.add(Dense(15, input_shape=(13,), activation='relu'))
 #model.add(Dense(50, activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [ ]:
# evaluate model with standardized dataset
#80% @ 200 epochs 15 neurons
estimators = []
estimators.append(('standardize', StandardScaler()))
estimator = KerasClassifier(model=create_baseline, epochs=200, batch_size=30, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

<ipython-input-37-3dc2b47adc8e>:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(model=create_baseline, epochs=200, batch_size=30, verbose=1)


AttributeError: ignored

In [ ]:
#84.30% @ 200 epochs with 20 neurons

In [ ]:
X_f = focused_heart.loc[:, 'age':'thall']
y_f = focused_heart.loc[:,'output']
Xf_train, Xf_test, yf_train, yf_test = train_test_split(X_f,y_f,random_state=104,test_size=0.25, shuffle=True)

In [ ]:
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
cvscores = []
for train, test in kfold.split(X_f, y_f):
 # create model
 model = Sequential()
 model.add(Dense(20, input_shape=(10,), activation='relu'))
 #model.add(Dense(10, activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 # Fit the model
 model.fit(Xf_train, yf_train, epochs=200, batch_size=10, verbose=0)
 # evaluate the model
 scores = model.evaluate(Xf_test, yf_test, verbose=0)
 print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
 cvscores.append(scores[1] * 100)

print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
def focused_nn():
 # create model
 model = Sequential()
 model.add(Dense(10, input_shape=(10,), activation='relu'))
 model.add(Dense(10, activation='relu'))
 model.add(Dense(1, activation='sigmoid'))
 # Compile model
 model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [ ]:
# evaluate model with standardized dataset
estimators_b = []
estimators_b.append(('standardize', StandardScaler()))
estimator = KerasClassifier(model=focused_nn, epochs=200, batch_size=30, verbose=1)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X_f, y_f, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(units = hp.Int('Dense1_neurons',min_value=4,max_value=32,step=2), activation= 'relu'))
  model.add(LayerNormalization())
  model.add(Dropout(rate = hp.Float('D1_rate',min_value=.10,max_value=0.20,step=0.05)))
  model.add(Dense(units = hp.Int('Dense2_neurons',min_value=2,max_value=20,step=2), activation= 'relu'))
  model.add(LayerNormalization())
  model.add(Dense(1, activation='sigmoid'))
 # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model_i = build_model

In [ ]:
import keras_tuner as kt

tuner = kt.GridSearch(model_j,
                     objective='val_accuracy',
                     max_trials=10,
                     directory='/content/drive/MyDrive/Colab Notebooks/model tuning',
                     project_name='RNN modele')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

tuner.search(Xf_train, yf_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print("Dense 1 nerons", best_hps.get('Dense1_neurons'))
print("Dense 2 nerons", best_hps.get('Dense2_neurons'))
print("D1 rate", best_hps.get('D1_rate'))

Dense 1 nerons 4
Dense 2 nerons 2
D1 rate 0.1


In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/20
6/6 [==============================] - 4s 57ms/step - loss: 0.7016 - accuracy: 0.5525 - val_loss: 0.7666 - val_accuracy: 0.4348
Epoch 2/20
6/6 [==============================] - 0s 15ms/step - loss: 0.6976 - accuracy: 0.5580 - val_loss: 0.7607 - val_accuracy: 0.4348
Epoch 3/20
6/6 [==============================] - 0s 12ms/step - loss: 0.6958 - accuracy: 0.5580 - val_loss: 0.7555 - val_accuracy: 0.4348
Epoch 4/20
6/6 [==============================] - 0s 13ms/step - loss: 0.6922 - accuracy: 0.5635 - val_loss: 0.7512 - val_accuracy: 0.4348
Epoch 5/20
6/6 [==============================] - 0s 15ms/step - loss: 0.6931 - accuracy: 0.5580 - val_loss: 0.7472 - val_accuracy: 0.4348
Epoch 6/20
6/6 [==============================] - 0s 11ms/step - loss: 0.6915 - accuracy: 0.5580 - val_loss: 0.7441 - val_accuracy: 0.4348
Epoch 7/20
6/6 [==============================] - 0s 12ms/step - loss: 0.6917 - accuracy: 0.5525 - val_loss: 0.7407 - val_accuracy: 0.4348
Epoch 8/20
6/6 [===========

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)